#Requirements

!pip install pandas==1.1.5
!pip install numpy==1.22.4
!pip install scipy==1.7.3
!pip install scikit-learn==1.1.1
!pip install joblib==1.1.0
!pip install pyyaml==5.4.1
!pip install shap==0.38.1
!pip install matplotlib==3.5.1
!pip install xgboost==1.6.1
!pip install joblib==1.1.0

#### To run this code:
1) Have a "data" folder with the dataset stored.

2) Have a "settings" folder with:

    a) A YAML file with the model, hyperparameters, list of numerical variables
    b) A YAML file with a list of the variable names, labels and descriptions (This is how you would like them to appear in a graph).
    c) A YAML file with two or more 'sample' patients. 
    
3) Check for names and parameters in the cell below

In [ ]:
import helpers_mi as helper
import pandas as pd
import numpy as np
from scipy import stats
import joblib

import yaml
from yaml.loader import SafeLoader

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline, FunctionTransformer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer


from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
    roc_auc_score,
)

import pickle


from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    ExtraTreesClassifier,
)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression

import shap 

import matplotlib.pyplot as plt
%matplotlib agg

### IMPORTANT INFORMATION

If you performed **MULTIPLE IMPUTATION** (MICE procedure):<br>
1. Keep the name without numbers in the 'data_file'. Numbers will be added later.<br>
&emsp;Example:  MICE was performed and it produced 50 files called data1.csv, data2.csv, etc. The 'data_file' should only be called data.csv.  <br>
2. Change the number of 'n_datasets' to the number of data files produced by MICE.<br> 
3. This code does NOT perform the MICE procedure. If your data is incomplete it will be imputed using IterativeImputer. <br>
    
If you want to perform **EXTERNAL-INTERNAL VALIDATION**:<br>
1. Change 'extint_val to True. <br>
2. Your dataset must have a 'cluster' variable with the samples identified to which cluster they belong to. Use numbers. <br>
3. In the 'settings.yaml' file, under 'clusters', name your clusters and add the identification number to them. This will allow for the clusters to be named. <br>
    
If you dont want to perform **EXTERNAL-INTERNAL VALIDATION**:<br>
1. Change 'extint_val' to False<br>
2. In the 'settings.yaml' file, under 'clusters', name your clusters Fold 1: 1, Fold 2: 2 and so on until Fold 5.<br>

In [ ]:
#Seed for reproducibility
np.random.seed(0)

#Dataset file. If you have multiple datasets DO NOT add the number. It will be added later. 
#Example if your files are called cluster_mi1, cluster_mi2, etc. just type cluster_mi as the name. 
data_file = f'data/mi_data/ndipass_period1_mi.csv'

#Change this value to the name of the id variable if you have any
id_var = 'id'

#Change this for the number of datasets your MICE procedure created
n_datasets = 50

#If the outcome needs imputing change this to True
impute_output = False

#If you have predetermined clusters for cross validation, this should be True
extint_val = False

#Name of the output variable you want to predict.
target_variable = 'ndipass'

#File to output the results
output_file       = 'data/results/results_models_development.csv'

#Pickle files to store results
train_save_file   = 'data/results/train_save_file.txt'
test_save_file    = 'data/results/test_save_file.txt'

train_save_file_merged   = 'data/results/train_save_file_merged.txt'
test_save_file_merged    = 'data/results/test_save_file_merged.txt'

#Settings files
settings_file     = 'settings/settings.yml'
sample_file       = 'settings/sample_patients.yml'
variable_names    = 'settings/variable_names.yml'

#Model save
model_dump = 'savedmodels/'

In [ ]:
helper.check_paths() #Check that all the folders needed are there

In [ ]:
#Settings file with models and their hyperparameters
with open(settings_file, "r") as stream:
    settings = yaml.load(stream, Loader=SafeLoader)

#Variable names and labels for graphs
with open(variable_names, "r") as stream:
    var_names = yaml.load(stream, Loader=SafeLoader)
    
#Example patients
with open(sample_file, 'r') as stream:
    samples = yaml.load(stream, Loader = SafeLoader)

In [ ]:
train_results, test_results = helper.get_results_dicts(settings)

In [ ]:
for d in range(n_datasets):
    
    #Getting the dataset for this run
    if n_datasets>1:
        dataset = data_file[:-4]
        dataset = dataset + f'{d+1}.csv'
    else:
        dataset = data_file
    df, x, y, clusters_idx = helper.get_data(dataset, target_variable, impute_output = impute_output, extint_val = extint_val, id_var=id_var)
    
    #Getting the sample patients
    sample_x = pd.DataFrame(columns = x.columns)

    for patient, values in samples.items():
        sample_x = sample_x.append(values, ignore_index = True)
        
        
    print(f"Running dataset N{d+1}")
    for model_name, attr in settings['models'].items():
        best_model = None
        best_pipe = None
        best_auc = 0
        best_thresh = 0


        print(model_name)
        proba_df_train = pd.DataFrame(columns = ['cluster','Train', 'Predicted', 'Proba'])
        proba_df_test = pd.DataFrame(columns = ['cluster','Test', 'Predicted', 'Proba'])
        
        numerical_features, categorical_features = helper.feature_discrimination(x,settings)

        sample_m = sample_x[x.columns].copy()

        #Changing the order of features to match it to the ones after the pipeline - This is done for SHAP purposes
        x_ = x.loc[:, numerical_features.to_list()+categorical_features.to_list()].copy()
        sample_m = sample_m.loc[:, numerical_features.to_list()+categorical_features.to_list()]

        #Declaring the pipeline
        numeric_pipeline = Pipeline(
            steps = [
                ("impute", IterativeImputer(sample_posterior = True, random_state = 0)),
                ("scale", MinMaxScaler()),
            ]
        )

        categorical_pipeline = Pipeline(
            steps = [
                ("impute", SimpleImputer(strategy = 'most_frequent')),
                ("one-hot", OneHotEncoder(handle_unknown = 'ignore', sparse = False, ))
            ]
        )
        preprocessor = ColumnTransformer(
            transformers = [
                ("Number", numeric_pipeline, numerical_features),
                ("Category", categorical_pipeline, categorical_features)

            ],
            remainder = 'passthrough')

        pipe = Pipeline(
            steps = [('preprocessor', preprocessor)])

        #Declaring the model
        model_string = helper.create_model(model_name, attr['param'])

        #List for example patients
        y_sample_test = []

        #Divide the datasets into train-test with respect to pre-defined clusters
        for cluster_name in settings['clusters']:
            n_cluster = settings['clusters'][cluster_name]
            print(f'Running Cluster N{n_cluster}, {cluster_name}')

            x_train = x_[clusters_idx != n_cluster].copy()
            y_train =  y[clusters_idx != n_cluster].copy()
            x_test  = x_[clusters_idx == n_cluster].copy()
            y_test  =  y[clusters_idx == n_cluster].copy()
            
            test_results[model_name][cluster_name]['shap_df'].append(x_test)

            x_train = pipe.fit_transform(x_train)
            x_test  = pipe.transform(x_test)
            sample_model = pipe.transform(sample_m)
            
            features = list(numerical_features) + list(pipe['preprocessor'].transformers_[1][1]['one-hot'].get_feature_names_out(categorical_features))
            x_train= pd.DataFrame(x_train, columns = features)
            x_test = pd.DataFrame(x_test, columns = features)
            sample_model = pd.DataFrame(sample_model, columns = features)
            
            #Instantiate the model
            model = eval(model_string)
            model.fit(x_train, y_train)

            #Compute SHAP if needed
            if (settings['models'][model_name]['shap'] == True) & (d == 0):
                print("Getting SHAP values")
                shap_type = settings['models'][model_name]['shap_type']
                test_results[model_name][cluster_name] = helper.compute_SHAP(model, 
                                                                             x_test,
                                                                             test_results[model_name][cluster_name],
                                                                             categorical_features,
                                                                             shap_type
                                                                            )


            y_proba_train = model.predict_proba(x_train)[:,1]
            y_proba_test  = model.predict_proba(x_test)[:,1]    

            y_sample_test.append(model.predict_proba(sample_model)[:,1])

            #Getting best threshold based on ROC curve 
            #Or calculate performance estimates based on predicted probabities
            thresh = helper.get_threshold(y_train, y_proba_train)
            y_pred_train = helper.get_prediction(y_train, y_proba_train, thresh)
            y_pred_test  = helper.get_prediction(y_test, y_proba_test, thresh)
            
            test_results[model_name][cluster_name] = helper.get_ROC_cluster_mi(model, 
                                                                               x_test, 
                                                                               y_test, 
                                                                               test_results, 
                                                                               model_name, 
                                                                               cluster_name)

            test_results[model_name][cluster_name] = helper.get_calibration(y_test,
                                                                            y_proba_test, 
                                                                            test_results,
                                                                            model_name,
                                                                            cluster_name)
            
            train_results[model_name][cluster_name] = helper.get_calibration(y_train,
                                                                             y_proba_train,
                                                                             train_results,
                                                                             model_name,
                                                                             cluster_name)
            
            score = ''
            for score_name, attr in settings['scoring_train'].items():
                score = helper.get_scoring(score_name, attr)
                score_value = eval(score)
                train_results[model_name][cluster_name][score_name].append(score_value)

                
                SE = helper.get_SE(score_value, y_train)
                train_results[model_name][cluster_name][score_name + '_SE'].append(SE)
                
                CI = helper.get_CI(SE, y_train.shape[0])
                train_results[model_name][cluster_name][score_name + '_CI'].append(CI)

            score = ''
            for score_name, attr in settings['scoring_test'].items():
                score = helper.get_scoring(score_name, attr)
                score_value = eval(score)
                test_results[model_name][cluster_name][score_name].append(score_value)

                SE = helper.get_SE(score_value, y_test)
                test_results[model_name][cluster_name][score_name + '_SE'].append(SE)
                
                CI = helper.get_CI(SE, y_test.shape[0])
                test_results[model_name][cluster_name][score_name + '_CI'].append(CI)
                
            if (settings['models'][model_name]['save'] == True) & (d == 0):
                current_auc = roc_auc_score(y_test,y_proba_test)
                if best_auc == 0:
                    best_thresh = thresh
                    best_auc = current_auc
                    best_model = model
                    best_pipe = pipe
                elif best_auc< current_auc:
                    best_auc = current_auc
                    best_model = model
                    best_pipe = pipe
                    best_thresh = thresh
                    

            proba_df = pd.DataFrame(columns = ['cluster','Train', 'Predicted', 'Proba'])
            proba_df['Train']     = y_train
            proba_df['Predicted'] = y_pred_train
            proba_df['Proba']     = y_proba_train
            proba_df['cluster']   = clusters_idx
#             proba_df.to_csv(f'data/Probabilities output/{model_name}_mi{d+1}_{cluster_name}_Train.csv', sep=';', index=False)
            train_results[model_name][cluster_name]['Proba_df'].append(proba_df)
            
            proba_df_train = pd.concat([proba_df_train, proba_df])
    
            proba_df = pd.DataFrame(columns = ['cluster','Test', 'Predicted', 'Proba'])
            proba_df['Test']      = y_test
            proba_df['Predicted'] = y_pred_test
            proba_df['Proba']     = y_proba_test
            proba_df['cluster']   = clusters_idx
#             proba_df.to_csv(f'data/Probabilities output/{model_name}_mi{d+1}_{cluster_name}_Test.csv', sep=';', index=False)
            test_results[model_name][cluster_name]['Proba_df'].append(proba_df)
            
            proba_df_test = pd.concat([proba_df_test, proba_df])

        if (settings['models'][model_name]['save'] == True) & (d == 0):
            full_model = Pipeline(
                steps = [('preprocessor', best_pipe),
                         ('predictor', best_model)])
            joblib.dump(full_model, str(model_dump + model_name + '_cv.joblib'))
            print('Best threshold is: ', best_thresh)
            
        sample_x[f"{model_name} "] = np.mean(y_sample_test, axis = 0)
        
#         proba_df_train.to_csv(f'data/Probabilities output/{model_name}_cluster_mi{d+1}_Proba_Train.csv', sep=';', index=False)
        proba_df_test.to_csv(f'data/Probabilities output/{model_name}_cluster_mi{d+1}_Proba_Test.csv', sep=';', index=False)
    
sample_x.to_csv('data/Samples.csv', sep=';')

In [ ]:
explainer = eval(str(shap_type + '(model = model)'))

In [ ]:
#Merging the results from the n-datasets
train_results_merged, test_results_merged = helper.merge_results(train_results, test_results, settings)

helper.save_results(train_save_file_merged, train_results_merged)
helper.save_results(test_save_file_merged,  test_results_merged)

In [ ]:
reform = {(outerKey, innerKey): values for outerKey, innerDict in test_results_merged.items() for innerKey, values in innerDict.items()}
rows = pd.MultiIndex.from_tuples(list(reform.keys()), names = ['Model','Region'])

to_keep = ['roc_auc','Intercept','Slope','precision','NPV']
columns = []
for item in to_keep:
    columns.append(item)
    columns.append(item + '_CI')
    columns.append(item + '_SE')


train_results_df = pd.DataFrame(columns = columns,
                                index = rows)
test_results_df = pd.DataFrame(columns = columns,
                               index = rows)

for model_name in settings['models']:
    for cluster_name in settings['clusters']:
        for item in to_keep:
            train_results_df.loc[(model_name,cluster_name),[item]] = round(train_results_merged[model_name][cluster_name][item][0],4)
            test_results_df.loc[(model_name,cluster_name),[item]] = round(test_results_merged[model_name][cluster_name][item][0],4)

            train_results_df.loc[(model_name,cluster_name),[item + '_SE']] = round(train_results_merged[model_name][cluster_name][item + '_SE'][0],4)
            test_results_df.loc[(model_name,cluster_name),[item + '_SE']] = round(test_results_merged[model_name][cluster_name][item + '_SE'][0],4)

            train_results_df.loc[(model_name,cluster_name),[item + '_CI']] = round(train_results_merged[model_name][cluster_name][item + '_CI'][0],4)
            test_results_df.loc[(model_name,cluster_name),[item + '_CI']] = round(test_results_merged[model_name][cluster_name][item + '_CI'][0],4)

    train_results_df.loc[(model_name,'Overall'),:] = round(train_results_df.loc[(model_name)].mean(),4)
    test_results_df.loc[(model_name,'Overall'),:] = round(test_results_df.loc[(model_name)].mean(),4)
            


In [ ]:
for model_name, attr in settings['models'].items():  
    #Comment this to avoid SHAP
    print(model_name)
    if settings['models'][model_name]['shap'] == True:
        print('Getting SHAP Plots')
#         test_results_merged[model_name] = helper.get_SHAP_plot_cluster_mi_merged(test_results_merged, model_name, var_names,settings, cols = 3)
        test_results[model_name] = helper.get_SHAP_plot_cluster_mi(test_results, model_name, var_names,settings, cols = 3)
    print('Getting ROC Plots')
    test_results_merged[model_name] = helper.get_ROC_plot_cluster(test_results_merged, test_results, model_name, settings)
    


In [ ]:
results_df = pd.concat([train_results_df, test_results_df], keys = ['Train', 'Test'], axis = 0)

In [ ]:
results_df = results_df.reset_index(level= [2])
results_df.columns.values[0] = 'Region'

results_df = results_df.reset_index(level= [1])
results_df.columns.values[0] = 'Model'

In [ ]:
results_df.to_csv(output_file, sep =';')